In [ ]:
import re
import random
import pandas as pd

In [ ]:
data = pd.read_csv('/content/df_nan_removed_temp.csv')

In [ ]:
data = data.drop(['Unnamed: 0'], axis = 1)
data = data.drop(['Unnamed: 0.1'], axis = 1)

In [ ]:
data = data[data['SYMPTOM INPUTS'].notna()]

In [ ]:
data = data[data['QUESTIONS ASKED'].notna()]

In [ ]:
data = data[data['OPTIONS GIVEN'].notna()]

In [ ]:
#data = data.head(300)

In [ ]:
data

,SYMPTOM INPUTS,QUESTIONS ASKED,OPTIONS GIVEN
0,Abdominal pain,Abdominal pain: when did it start?,Abdominal pain:Abdominal pain
1,Abdominal pain,Abdominal pain: select the area you feel pain ...,Abdominal pain:today
2,Abdominal pain,Abdominal pain: select the area you feel pain ...,Abdominal pain: in the last week
3,Abdominal pain,Abdominal pain: select the area you feel pain ...,Abdominal pain: in last month
4,Abdominal pain,Abdominal pain: select the area you feel pain ...,Abdominal pain: 1-3 month ago
...,...,...,...
1839,groin swelling,groin swelling: do you have trouble breathing ...,groin swelling: yes
1840,groin swelling,groin swelling: do you have trouble breathing ...,groin swelling: no
1851,Goiter,"Goiter: does you heart race, flutter or skip b...",Goiter:Goiter
1852,Goiter,"Goiter: is your arm quivering, trembling, or s...",Goiter: yes


In [ ]:
#data.to_csv("csv_data.csv")

In [ ]:
def decontractions(phrase):
    """decontracted takes text and convert contractions into natural form.
     ref: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python/47091490#47091490"""
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"won\’t", "will not", phrase)
    phrase = re.sub(r"can\’t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)

    phrase = re.sub(r"n\’t", " not", phrase)
    phrase = re.sub(r"\’re", " are", phrase)
    phrase = re.sub(r"\’s", " is", phrase)
    phrase = re.sub(r"\’d", " would", phrase)
    phrase = re.sub(r"\’ll", " will", phrase)
    phrase = re.sub(r"\’t", " not", phrase)
    phrase = re.sub(r"\’ve", " have", phrase)
    phrase = re.sub(r"\’m", " am", phrase)

    return phrase



def preprocess(text):
    # convert all the text into lower letters
    # remove the words betweent brakets ()
    # remove these characters: {'$', ')', '?', '"', '’', '.',  '°', '!', ';', '/', "'", '€', '%', ':', ',', '('}
    # replace these spl characters with space: '\u200b', '\xa0', '-', '/'
    
    text = text.lower()
    text = decontractions(text)
    text = re.sub('[$)\?"’.°!;\'€%:,(/]', '', text)
    text = re.sub('\u200b', ' ', text)
    text = re.sub('\xa0', ' ', text)
    text = re.sub('-', ' ', text)
    return text

In [ ]:
data['preprocessed_question'] = data['OPTIONS GIVEN'].apply(preprocess)
data['preprocessed_answer'] = data['QUESTIONS ASKED'].apply(preprocess)
data.head()

,SYMPTOM INPUTS,QUESTIONS ASKED,OPTIONS GIVEN,preprocessed_question,preprocessed_answer
0,Abdominal pain,Abdominal pain: when did it start?,Abdominal pain:Abdominal pain,abdominal painabdominal pain,abdominal pain when did it start
1,Abdominal pain,Abdominal pain: select the area you feel pain ...,Abdominal pain:today,abdominal paintoday,abdominal pain select the area you feel pain u...
2,Abdominal pain,Abdominal pain: select the area you feel pain ...,Abdominal pain: in the last week,abdominal pain in the last week,abdominal pain select the area you feel pain u...
3,Abdominal pain,Abdominal pain: select the area you feel pain ...,Abdominal pain: in last month,abdominal pain in last month,abdominal pain select the area you feel pain u...
4,Abdominal pain,Abdominal pain: select the area you feel pain ...,Abdominal pain: 1-3 month ago,abdominal pain 1 3 month ago,abdominal pain select the area you feel pain u...


In [ ]:
import numpy as np

input_docs = []
target_docs = []
input_tokens = set()
target_tokens = set()

In [ ]:
for ind in data.index:
  input_doc, target_doc = data['preprocessed_question'][ind], data['preprocessed_answer'][ind]
  #print(data['preprocessed_question'][ind], data['preprocessed_answer'][ind])
  #print(input_doc, target_doc)

  # Appending each input sentence to input_docs
  input_docs.append(input_doc)
  # Splitting words from punctuation 
  target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc))
  # Redefine target_doc below and append it to target_docs
  target_doc = '<START> ' + target_doc + ' <END>'
  target_docs.append(target_doc)

  # Now we split up each sentence into words and add each unique word to our vocabulary set
  for token in re.findall(r"[\w']+|[^\s\w]", input_doc):
    if token not in input_tokens:
      input_tokens.add(token)
  for token in target_doc.split():
    if token not in target_tokens:
      target_tokens.add(token)


input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

input_features_dict = dict(
    [(token, i) for i, token in enumerate(input_tokens)])
target_features_dict = dict(
    [(token, i) for i, token in enumerate(target_tokens)])

reverse_input_features_dict = dict(
    (i, token) for token, i in input_features_dict.items())
reverse_target_features_dict = dict(
    (i, token) for token, i in target_features_dict.items())


In [ ]:
input_docs

['abdominal painabdominal pain',
 'abdominal paintoday',
 'abdominal pain in the last week',
 'abdominal pain in last month',
 'abdominal pain 1 3 month ago',
 'abdominal pain over 3 months ago',
 'abdominal pain year ago',
 'abdominal pain point at diagram',
 'abdominal pain right',
 'abdominal pain left',
 'abdominal pain both sides',
 'abdominal pain i am not sure',
 'abdominal pain changing position',
 'abdominal pain cough',
 'abdominal pain eating',
 'abdominal pain deep breathing',
 'abdominal pain  exposure to cold',
 'abdominal pain going to bathroom',
 'abdominal pain hunger',
 'abdominal pain lying down',
 'abdominal pain physical activity',
 'abdominal painpressing on area',
 'abdominal pain sitting down',
 'abdominal pain standing',
 'abdominal pain walking',
 'abdominal pain dairy intake',
 'abdominal pain eating suspect food',
 'abdominal pain going to bathroom',
 'abdominal pain intercourse',
 'abdominal pain possible food allergy',
 'abdominal pain recent childbirth',


In [ ]:
target_docs

['<START> abdominal pain when did it start <END>',
 '<START> abdominal pain select the area you feel pain using diagram <END>',
 '<START> abdominal pain select the area you feel pain using diagram <END>',
 '<START> abdominal pain select the area you feel pain using diagram <END>',
 '<START> abdominal pain select the area you feel pain using diagram <END>',
 '<START> abdominal pain select the area you feel pain using diagram <END>',
 '<START> abdominal pain select the area you feel pain using diagram <END>',
 '<START> abdominal pain which side is bothering you <END>',
 '<START> abdominal pain what makes it worse <END>',
 '<START> abdominal pain what makes it worse <END>',
 '<START> abdominal pain what makes it worse <END>',
 '<START> abdominal pain what makes it worse <END>',
 '<START> abdominal pain do you think any of these triggers pain <END>',
 '<START> abdominal pain do you think any of these triggers pain <END>',
 '<START> abdominal pain do you think any of these triggers pain <EN

In [ ]:
max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])
max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])


encoder_input_data = np.zeros(
    (len(input_docs), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for line, (input_doc, target_doc) in enumerate(zip(input_docs, target_docs)):
    for timestep, token in enumerate(re.findall(r"[\w']+|[^\s\w]", input_doc)):
        #Assign 1. for the current line, timestep, & word in encoder_input_data
        encoder_input_data[line, timestep, input_features_dict[token]] = 1.
    
    for timestep, token in enumerate(target_doc.split()):
        decoder_input_data[line, timestep, target_features_dict[token]] = 1.
        if timestep > 0:
            decoder_target_data[line, timestep - 1, target_features_dict[token]] = 1.

In [ ]:
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model
#Dimensionality
dimensionality = 256
#The batch size and number of epochs
batch_size = 10
epochs = 100
#Encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(dimensionality, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]
#Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(dimensionality, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
#Model
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
#Compiling
training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')

In [ ]:
#Training
training_model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)
training_model.save('training_model.h5')

Epoch 1/100
106/106 [==============================] - 19s 149ms/step - loss: 1.6426 - accuracy: 0.6848 - val_loss: 1.7101 - val_accuracy: 0.7165
Epoch 2/100
106/106 [==============================] - 15s 139ms/step - loss: 1.5092 - accuracy: 0.7059 - val_loss: 1.7604 - val_accuracy: 0.7175
Epoch 3/100
106/106 [==============================] - 15s 139ms/step - loss: 1.4569 - accuracy: 0.7091 - val_loss: 1.7985 - val_accuracy: 0.7202
Epoch 4/100
106/106 [==============================] - 15s 138ms/step - loss: 1.4186 - accuracy: 0.7144 - val_loss: 1.8362 - val_accuracy: 0.7232
Epoch 5/100
106/106 [==============================] - 15s 140ms/step - loss: 1.3732 - accuracy: 0.7272 - val_loss: 1.8502 - val_accuracy: 0.7351
Epoch 6/100
106/106 [==============================] - 15s 139ms/step - loss: 1.3266 - accuracy: 0.7373 - val_loss: 1.8352 - val_accuracy: 0.7407
Epoch 7/100
106/106 [==============================] - 14s 136ms/step - loss: 1.2785 - accuracy: 0.7502 - val_loss: 1.8439 -

In [ ]:
from keras.models import load_model
training_model = load_model('training_model.h5')
encoder_inputs = training_model.input[0]
encoder_outputs, state_h_enc, state_c_enc = training_model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

latent_dim = 256
decoder_state_input_hidden = Input(shape=(latent_dim,))
decoder_state_input_cell = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]
decoder_outputs, state_hidden, state_cell = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_hidden, state_cell]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)


In [ ]:
def decode_response(test_input):
    #Getting the output states to pass into the decoder
    states_value = encoder_model.predict(test_input)
    #Generating empty target sequence of length 1
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    #Setting the first token of target sequence with the start token
    target_seq[0, 0, target_features_dict['<START>']] = 1.
    
    #A variable to store our response word by word
    decoded_sentence = ''
    
    stop_condition = False
    while not stop_condition:
          #Predicting output tokens with probabilities and states
          output_tokens, hidden_state, cell_state = decoder_model.predict([target_seq] + states_value)
    #Choosing the one with highest probability
          sampled_token_index = np.argmax(output_tokens[0, -1, :])
          sampled_token = reverse_target_features_dict[sampled_token_index]
          decoded_sentence += " " + sampled_token
    #Stop if hit max length or found the stop token
          if (sampled_token == '<END>' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
    #Update the target sequence
          target_seq = np.zeros((1, 1, num_decoder_tokens))
          target_seq[0, 0, sampled_token_index] = 1.
          #Update states
          states_value = [hidden_state, cell_state]
    return decoded_sentence

In [ ]:
class ChatBot:
  negative_responses = ("no", "nope", "nah", "naw", "not a chance", "sorry")
  exit_commands = ("quit", "pause", "exit", "goodbye", "bye", "later", "stop")
#Method to start the conversation
  def start_chat(self):
    user_response = input("Hi, I'm a chatbot trained on health data. Would you like to chat with me?\n")
    
    if user_response in self.negative_responses:
      print("Ok, have a great day!")
      return
    self.chat(user_response)
#Method to handle the conversation
  def chat(self, reply):
    while not self.make_exit(reply):
      reply = input(self.generate_response(reply)+"\n")
    
  #Method to convert user input into a matrix
  def string_to_matrix(self, user_input):
    tokens = re.findall(r"[\w']+|[^\s\w]", user_input)
    user_input_matrix = np.zeros(
      (1, max_encoder_seq_length, num_encoder_tokens),
      dtype='float32')
    for timestep, token in enumerate(tokens):
      if token in input_features_dict:
        user_input_matrix[0, timestep, input_features_dict[token]] = 1.
    return user_input_matrix
  
  #Method that will create a response using seq2seq model we built
  def generate_response(self, user_input):
    input_matrix = self.string_to_matrix(user_input)
    chatbot_response = decode_response(input_matrix)
    #Remove <START> and <END> tokens from chatbot_response
    chatbot_response = chatbot_response.replace("<START>",'')
    chatbot_response = chatbot_response.replace("<END>",'')
    return chatbot_response
#Method to check for exit commands
  def make_exit(self, reply):
    for exit_command in self.exit_commands:
      if exit_command in reply:
        print("Ok, have a great day!")
        return True
    return False

In [ ]:
chatbot = ChatBot()


In [ ]:
chatbot.start_chat()

Hi, I'm a chatbot trained on health data. Would you like to chat with me?
abdominal pain
 abdominal pain did it start gradually
abdominal pain: yes
 abdominal pain are you more gassy
abdominal pain: No
 abdominal pain did it start gradually
abdominal pain: no
 abdominal pain are you more gassy
abdominal pain: yes
 abdominal pain are you more gassy
abdominal pain: no


KeyboardInterrupt: ignored

In [ ]:
#df = pd.read_csv('/content/df_nan_removed - Copy.csv')

In [ ]:
#df

,Unnamed: 0,SYMPTOM INPUTS,QUESTIONS ASKED,OPTIONS GIVEN
0,0.0,Abdominal pain,Abdominal pain: when did it start?,Abdominal pain
1,1.0,Abdominal pain,Abdominal pain: select the area you feel pain ...,today
2,2.0,Abdominal pain,Abdominal pain: select the area you feel pain ...,in the last week
3,3.0,Abdominal pain,Abdominal pain: select the area you feel pain ...,in last month
4,4.0,Abdominal pain,Abdominal pain: select the area you feel pain ...,1-3 month ago
...,...,...,...,...
1849,NaN,NaN,NaN,NaN
1850,NaN,NaN,NaN,NaN
1851,4127.0,Goiter,"Goiter: does you heart race, flutter or skip b...",Goiter
1852,4132.0,Goiter,"Goiter: is your arm quivering, trembling, or s...",yes


In [ ]:
#df['OPTIONS GIVEN'] =  df['SYMPTOM INPUTS'] + ':' + df['OPTIONS GIVEN'].astype(str) 

In [ ]:
#df.to_csv("df_nan_removed_temp.csv")